In [1]:
import tengp
from experiment_settings import ckorns12_funset
import symreg

In [2]:
exact_solution = [0, 0, 0, 2,
                  0, 0, 0, 2.1,
                  0, 0, 0, 9.8,
                  3, 7, 3, 0,
                  6, 8, 0, 0,
                  3, 6, 9, 0,
                  0, 0, 0, 1.3,
                  3, 2, 11, 0,
                  5, 12, 0, 0,
                  3, 10, 13, 0,
                  2, 5, 14, 0,
                  15]

In [3]:
cparamsKorns = tengp.Parameters(5, 1, 1, 11, ckorns12_funset, real_valued=True, constants=10)

bounds = tengp.individual.IndividualBuilder(cparamsKorns).create().bounds

individual = cparamsKorns.individual_class(exact_solution, bounds, cparamsKorns)

In [4]:
import random

x_train, y_train, x_test, y_test = symreg.get_benchmark_korns(random, 12)

y_pred = individual.transform(x_train)

In [5]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_pred, y_train)

4.34501135329273e-30

# Try to optimize with PSO

In [6]:
import pygmo as pg

class cost_function:
    def __init__(self, X, Y, params, bounds):
        self.params = params
        self.bounds = bounds
        self.X = X
        self.Y = Y
    
    def fitness(self, x):      
        individual = tengp.individual.NPIndividual(list(x), self.bounds, self.params)
        
        pred = individual.transform(self.X)
        
        try:
            return [mean_squared_error(pred, self.Y)]
        except ValueError:
            return [10000000000]
        
    def get_bounds(self):
        self.bounds[0][-1] = 12
        return self.bounds

In [8]:
prob = pg.problem(cost_function(x_train, y_train, cparamsKorns, bounds))



algo = pg.algorithm(pg.pso(gen=250))

algo.set_verbosity(2)

pop = pg.population(prob, 20)

pop = algo.evolve(pop)

print(pop.champion_f[0])

/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:35: RuntimeWarning: invalid value encountered in sqrt
  return  np.sqrt(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:38: RuntimeWarning: overflow encountered in exp
  return np.exp(x)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:133: RuntimeWarning: overflow encountered in multiply
  value = (1-coeff)*self.nodes[lower].value + coeff*self.nodes[upper].value
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:11: RuntimeWarning: invalid value encountered in greater
  return np.log(x, out=np.copy(x), where=x>0)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:103: RuntimeWarning: invalid value encountered in multiply
  L = (1-b)*x_l1 + b*x_u1
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:107: RuntimeWarning: invalid value encountered in multiply
  U = (1-c)*x_l2 + c*x_u2
/mnt/c/Users/jarinov-notas/dev/tengp/t

1.1103344678680787


# Random walk

In [45]:
import random

random.seed(42)

STEP_SIZE = 0.1

NUMBER_OF_STEPS = 1000

start_point = exact_solution
random_walk = [start_point]
new_point = start_point[:]

i = 0
while i < NUMBER_OF_STEPS:
    #print(i, end=',')
    index = random.randint(0, len(start_point) - 1)
    
    move = random.choice([-STEP_SIZE, STEP_SIZE])  
    
    if (new_point[index] + move) < 0 or (new_point[index] + move) > bounds[1][index]:        
        continue
    
    
    new_point[index] += move
    
    
    if new_point not in random_walk:    
        random_walk.append(new_point)
        new_point = new_point[:]
        i += 1
    
    

In [46]:
fitnesses = []

for genes in random_walk:
    individual = cparamsKorns.individual_class(genes, bounds, cparamsKorns)
    output = individual.transform(x_train)
    fitnesses.append(mean_squared_error(output, y_train))

In [47]:
fitnesses

[4.34501135329273e-30,
 0.04441509539221854,
 0.04441509539221854,
 0.0926816901248828,
 1.6671840497655663,
 1.667115886497464,
 0.09275233610997505,
 0.16627955963277347,
 0.03960686195849194,
 0.16627955963277347,
 0.16627955963277347,
 0.16627955963277347,
 0.16627955963277347,
 0.07904143347349317,
 0.11088492662739688,
 0.1744240006012205,
 1.5940916835237218,
 1.6805467565713135,
 1.7173514489405082,
 1.7173514489405082,
 631.84882151488,
 779.8262437593214,
 646983.4912563058,
 96930200.7846127,
 96930200.7846127,
 336019494.3851184,
 270075603.2844795,
 270075603.2844795,
 270077499.4519053,
 270077499.4519053,
 270077499.4519053,
 333379660.7909674,
 333379660.7909674,
 333379660.7909674,
 333379660.7909674,
 256018593.26096722,
 189244845.4310236,
 250028473.28284287,
 250028473.28284287,
 226950617.17175102,
 177580424.7530429,
 177580424.7530429,
 710230786.0287775,
 1495367552.3727305,
 373785957.5309329,
 373785957.5309329,
 373785957.5309329,
 373785552.18922293,
 41143

In [48]:
from sklearn.metrics.pairwise import euclidean_distances

distances = euclidean_distances(random_walk, [start_point])

distances = distances.flatten()

In [49]:
import matplotlib.pyplot as plt
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [50]:
p = figure(plot_width=800, plot_height=400, title='random walk fitnesses')
x_axis = list(range(len(fitnesses)))
p.line(, fitnesses)

show(p)

In [52]:
fitnesses

[4.34501135329273e-30,
 0.04441509539221854,
 0.04441509539221854,
 0.0926816901248828,
 1.6671840497655663,
 1.667115886497464,
 0.09275233610997505,
 0.16627955963277347,
 0.03960686195849194,
 0.16627955963277347,
 0.16627955963277347,
 0.16627955963277347,
 0.16627955963277347,
 0.07904143347349317,
 0.11088492662739688,
 0.1744240006012205,
 1.5940916835237218,
 1.6805467565713135,
 1.7173514489405082,
 1.7173514489405082,
 631.84882151488,
 779.8262437593214,
 646983.4912563058,
 96930200.7846127,
 96930200.7846127,
 336019494.3851184,
 270075603.2844795,
 270075603.2844795,
 270077499.4519053,
 270077499.4519053,
 270077499.4519053,
 333379660.7909674,
 333379660.7909674,
 333379660.7909674,
 333379660.7909674,
 256018593.26096722,
 189244845.4310236,
 250028473.28284287,
 250028473.28284287,
 226950617.17175102,
 177580424.7530429,
 177580424.7530429,
 710230786.0287775,
 1495367552.3727305,
 373785957.5309329,
 373785957.5309329,
 373785957.5309329,
 373785552.18922293,
 41143

In [ ]:
p = figure(plot_width=800, plot_height=400, title='random walk distance from start point')
p.line(list(range(len(fitnesses))), distances)
show(p)